In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
os.chdir(r'C:\Users\Administrator\Desktop\02.Analyses')
from myfuncs import *

# Carregando dados na memoria

### Carregando dados do Enem

In [3]:
os.chdir(r'C:\Users\Administrator\Desktop\Neoway\Educação - Fontes Externas\ENEM\enem_2017\DADOS')

##### Selecionando subset de colunas para carregar

In [4]:
rTemp = pd.read_csv(r'MICRODADOS_ENEM_2017.csv',sep = ';',encoding= 'latin-1',nrows = 0)
rTemp2 = rTemp.head().T.reset_index()
selected_columns = ['NU_INSCRICAO','CO_PROVA_MT','CO_MUNICIPIO_PROVA','NO_MUNICIPIO_PROVA','CO_ESCOLA','CO_MUNICIPIO_ESC','NO_MUNICIPIO_ESC','SG_UF_ESC','TX_RESPOSTAS_MT',
                   'TX_GABARITO_MT']
use_cols = {}
for i,c in zip(rTemp2.index,rTemp2['index'].values):
    if c in selected_columns:
        use_cols[i] = c

In [5]:
rEnem = pd.read_csv(r'MICRODADOS_ENEM_2017.csv',sep = ';',encoding= 'latin-1',usecols = use_cols)

### Carregando de-para de cores de provas e questoes 

In [6]:
os.chdir(r'C:\Users\Administrator\Desktop\01.Data')
rColor = pd.read_excel(r'Referência cores ENEM 2017 (1).xlsx')

# Manipulacao de dados

## Extraindo acertos por questao
- A base do ENEM possui uma coluna descrevendo as alternativas assinaladas em forma de uma string sequencial
- Cada sequencia de string é ordenada de acordo com a disposicao das questoes determinada pela cor da prova
- Precisamos fazer o **reshuffle** das questões e normalizar a ordem

### Enriquecendo com infos de cores

##### Limpando base de infos de cores 

In [7]:
tColor = rColor.copy()

#elimina header
tColor = tColor.drop(labels = 0,axis = 0).reset_index(drop= True)
#elimina coluna extra
tColor.drop("CÓDIGO",axis = 1,inplace=True)
tColor

,403,404,406,405
0,1,9,18,5
1,2,10,19,6
2,3,11,20,7
3,4,1,5,12
4,5,2,6,13
5,6,3,7,14
6,7,4,8,15
7,8,14,11,1
8,9,15,12,2
9,10,16,13,3


##### Criando dicionario de-para 

In [19]:
color_dict = {}

#extrai lista de valores base para realizar reshuffle 
ref = tColor[403].values

#itera sobre as diferentes cores de provas e cria de-para com valores base
for c in [403,404,405,406]:
    qs = tColor[c].values
    color_dict[c] = {}
    for q,r in zip(qs,ref):
        color_dict[c][q] = r

##### Extraindo informacoes de acerto questao a questao na ordem normalizada entre provas (colunas `CHECK_#`)

In [37]:
tQ = rEnem.copy()

#filtra apenas de oferecimento regular - remove reoferecimentos
tQ = tQ[tQ['CO_PROVA_MT'].isin([403,404,405,406])]
#filtra colunas de indexacao,strings de respostas e gabarito
tQ = tQ[['NU_INSCRICAO','CO_ESCOLA','CO_PROVA_MT','TX_RESPOSTAS_MT','TX_GABARITO_MT']]
#inicializa o dataset reshuffled com o subset da prova base
tQ_reshuffled = pd.DataFrame()

#itera sobre as cores restantes e indexa de acordo com o reshuffle
for color in [403,404,405,406]:
    print("PROCESSING TESTS %s" %color)
    tQ_c =  tQ[tQ['CO_PROVA_MT'] == color]
    for i in range(0,45):
        print("%s" %i, end = " | ")
        q_no = color_dict[color][i+1]
        tQ_c["CHECK_"+str(q_no)] = tQ_c['TX_RESPOSTAS_MT'].str[i] == tQ_c['TX_GABARITO_MT'].str[i]
    print()
    tQ_reshuffled = tQ_reshuffled.append(tQ_c)

403 |    PROCESSING QUESTION 0


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


   PROCESSING QUESTION 1
   PROCESSING QUESTION 2
   PROCESSING QUESTION 3
   PROCESSING QUESTION 4
   PROCESSING QUESTION 5
   PROCESSING QUESTION 6
   PROCESSING QUESTION 7
   PROCESSING QUESTION 8
   PROCESSING QUESTION 9
   PROCESSING QUESTION 10
   PROCESSING QUESTION 11
   PROCESSING QUESTION 12
   PROCESSING QUESTION 13
   PROCESSING QUESTION 14
   PROCESSING QUESTION 15
   PROCESSING QUESTION 16
   PROCESSING QUESTION 17
   PROCESSING QUESTION 18
   PROCESSING QUESTION 19
   PROCESSING QUESTION 20
   PROCESSING QUESTION 21
   PROCESSING QUESTION 22
   PROCESSING QUESTION 23
   PROCESSING QUESTION 24
   PROCESSING QUESTION 25
   PROCESSING QUESTION 26
   PROCESSING QUESTION 27
   PROCESSING QUESTION 28
   PROCESSING QUESTION 29
   PROCESSING QUESTION 30
   PROCESSING QUESTION 31
   PROCESSING QUESTION 32
   PROCESSING QUESTION 33
   PROCESSING QUESTION 34
   PROCESSING QUESTION 35
   PROCESSING QUESTION 36
   PROCESSING QUESTION 37
   PROCESSING QUESTION 38
   PROCESSING QUESTIO

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:6211: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


405 |    PROCESSING QUESTION 0
   PROCESSING QUESTION 1
   PROCESSING QUESTION 2
   PROCESSING QUESTION 3
   PROCESSING QUESTION 4
   PROCESSING QUESTION 5
   PROCESSING QUESTION 6
   PROCESSING QUESTION 7
   PROCESSING QUESTION 8
   PROCESSING QUESTION 9
   PROCESSING QUESTION 10
   PROCESSING QUESTION 11
   PROCESSING QUESTION 12
   PROCESSING QUESTION 13
   PROCESSING QUESTION 14
   PROCESSING QUESTION 15
   PROCESSING QUESTION 16
   PROCESSING QUESTION 17
   PROCESSING QUESTION 18
   PROCESSING QUESTION 19
   PROCESSING QUESTION 20
   PROCESSING QUESTION 21
   PROCESSING QUESTION 22
   PROCESSING QUESTION 23
   PROCESSING QUESTION 24
   PROCESSING QUESTION 25
   PROCESSING QUESTION 26
   PROCESSING QUESTION 27
   PROCESSING QUESTION 28
   PROCESSING QUESTION 29
   PROCESSING QUESTION 30
   PROCESSING QUESTION 31
   PROCESSING QUESTION 32
   PROCESSING QUESTION 33
   PROCESSING QUESTION 34
   PROCESSING QUESTION 35
   PROCESSING QUESTION 36
   PROCESSING QUESTION 37
   PROCESSING QU

##### Calcula a quantidade de acertos totais por aluno para verificar calculo de acertos questao a questao 

In [38]:
tQ_reshuffled["ACERTOS"] = tQ_reshuffled.filter(regex = 'CHECK').sum(axis = 1)

In [43]:
tQ_reshuffled["ACERTOS"].value_counts()

10    489252
11    469206
9     461897
12    417502
8     390079
13    345802
7     290784
14    273876
15    208679
6     187812
16    156276
17    117467
5     102169
18     88483
19     68292
20     53926
4      46464
21     43190
22     35354
23     29295
24     24480
25     20591
26     17395
3      16588
27     14274
28     12022
29     10063
30      8319
31      6730
32      5661
2       4396
33      4380
34      3319
35      2578
36      1933
37      1439
0       1034
1       1000
38       950
39       646
40       434
41       235
42       130
43        73
44        32
45        10
Name: ACERTOS, dtype: int64

In [39]:
for c in tQ_reshuffled.filter(regex = 'CHECK').columns:
    tQ_reshuffled[c]=tQ_reshuffled[c].astype('int') 

In [40]:
tQ_reshuffled.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4434517 entries, 9 to 6731339
Data columns (total 51 columns):
CHECK_1            int32
CHECK_10           int32
CHECK_11           int32
CHECK_12           int32
CHECK_13           int32
CHECK_14           int32
CHECK_15           int32
CHECK_16           int32
CHECK_17           int32
CHECK_18           int32
CHECK_19           int32
CHECK_2            int32
CHECK_20           int32
CHECK_21           int32
CHECK_22           int32
CHECK_23           int32
CHECK_24           int32
CHECK_25           int32
CHECK_26           int32
CHECK_27           int32
CHECK_28           int32
CHECK_29           int32
CHECK_3            int32
CHECK_30           int32
CHECK_31           int32
CHECK_32           int32
CHECK_33           int32
CHECK_34           int32
CHECK_35           int32
CHECK_36           int32
CHECK_37           int32
CHECK_38           int32
CHECK_39           int32
CHECK_4            int32
CHECK_40           int32
CHECK_41    

##### Exportando dados para o disco

In [41]:
tQ_reshuffled.to_csv('ENEM_ACERTOS_QUESTOES_20180912_0300.csv',sep = ';',encoding='latin-1',index = False) 

In [46]:
tQ_reshuffled['CO_ESCOLA'].nunique()

32110

## Analisando a quantidade de preenchimentos por municipio
- Para a nossa análise, necessitamos das chaves de código de escola
- Na base de ENEM, cerca de 20% das observacoes nao possuem o preenchimento do codigo escola
- Faremos portanto um corte geografico da base que possua um numero absoluto de observacoes relevante, e uma % de preenchimento alta em relacao a base 

In [42]:
THRESH = 10000

tPrec = rEnem.copy()
gPrec = tPrec.groupby("NO_MUNICIPIO_PROVA").agg({'NU_INSCRICAO':'count','CO_ESCOLA':'count'})
gPrec = gPrec[gPrec['CO_ESCOLA'] > THRESH]
gPrec['perc_preench'] = gPrec['CO_ESCOLA']/gPrec['NU_INSCRICAO']

gPrec.sort_values('perc_preench',ascending = False)

,NU_INSCRICAO,CO_ESCOLA,perc_preench
NO_MUNICIPIO_PROVA,,,
São Paulo,302111,96539,0.319548
Curitiba,63038,19006,0.301501
Guarulhos,35821,10698,0.298652
Rio de Janeiro,171221,48662,0.284206
Fortaleza,125744,33709,0.268076
Porto Alegre,41480,10846,0.261475
Brasília,125257,32498,0.259451
Goiânia,68434,15864,0.231815
Manaus,120793,26522,0.219566


##### Escolheremos o municipio de `Sao Paulo`